In [4]:
import spacy
from spacy.training import offsets_to_biluo_tags

print("🔍 Prüfe auf nicht ausrichtbare Entities...")
for i, (text, ann) in enumerate(train_data):
    doc = nlp.make_doc(text)
    tags = offsets_to_biluo_tags(doc, ann["entities"])
    if "-" in tags:
        print(f"❌ Fehlerhafte Entity-Ausrichtung in Beispiel {i}:")
        for start, end, label in ann["entities"]:
            print(f"  → '{text[start:end]}' ({label}) [{start}:{end}]")



text = train_data[2][0]  # Beispiel 2 aus deiner Liste
entities = train_data[2][1]["entities"]
doc = nlp.make_doc(text)

tags = offsets_to_biluo_tags(doc, entities)
print("Tokens + Tags:")
for token, tag in zip(doc, tags):
    print(f"{token.text:20} {tag}")

🔍 Prüfe auf nicht ausrichtbare Entities...
❌ Fehlerhafte Entity-Ausrichtung in Beispiel 2:
  → 'SSE>> <<HAUSNUMMER>>' (STRASSE) [356:376]
  → '<<EMAIL>>
<mailto:' (EMAIL) [81:99]
  → '> gewählt. Da wir ' (EMAIL) [108:126]
  → 'geteilt, dass ich
' (EMAIL) [834:852]
  → ' <mailto:<<EMAIL>>' (EMAIL) [861:879]
  → '<<EMAIL>> <mailto:' (EMAIL) [1056:1074]
  → '>
bedanke ich mich' (EMAIL) [1083:1101]
  → '>>
Zählernummer' (ZÄHLERNUMMER) [438:453]
  → 'TZAHL>> <<W' (WOHNORT) [387:398]
  → 'skonto:
<' (VERTRAGSNUMMER) [414:423]
  → 'NAME>>' (VORNAME) [343:349]
  → 'NAME>>' (VORNAME) [1155:1161]
  → '<<STR' (NACHNAME) [350:355]
  → 'B.A.
' (NACHNAME) [1162:1167]
  → 'OSTLE' (POSTLEITZAHL) [381:386]
  → ' <' (HAUSNUMMER) [377:379]
❌ Fehlerhafte Entity-Ausrichtung in Beispiel 3:
  → '456123-5007643' (ZÄHLERNUMMER) [199:213]
  → '409291555' (VERTRAGSNUMMER) [177:186]
  → 'Nerger' (VORNAME) [164:170]
  → 'Grauel' (NACHNAME) [171:177]
❌ Fehlerhafte Entity-Ausrichtung in Beispiel 5:
  → '30 Euro' (ZA

In [2]:
# 🧩 Schritt 1: Imports und Setup
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import json
import random
from pathlib import Path
from spacy.util import minibatch
!python -m spacy download de_core_news_md
!pip install spacy-lookups-data



# 📁 Schritt 2: Funktion zum Laden von Daten aus JSON
def load_data_from_json(path):
    with open(path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    if isinstance(raw_data, dict):
        raw_data = [raw_data]

    TRAIN_DATA = []
    for entry in raw_data:
        text = entry["text"]
        entities = [(label["start"], label["end"], label["label"]) for label in entry["labels"]]
        TRAIN_DATA.append((text, {"entities": entities}))
    return TRAIN_DATA

# 🔄 Lade Trainings- und Dev-Daten separat
train_data = load_data_from_json("../../../data/original/granular_dataset_split_norm_cleaned/train_norm_cleaned.json")
dev_data = load_data_from_json("../../../data/original/granular_dataset_split_norm_cleaned/validation_norm_cleaned.json")
print(f"📥 Trainingsbeispiele: {len(train_data)}, Dev-Beispiele: {len(dev_data)}")

# 🧠 Schritt 3: Lade spaCy-Basismodell
base_model = "de_core_news_md"
nlp = spacy.load(base_model)


# Stelle sicher, dass NER-Komponente existiert
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Registriere alle Labels aus beiden Datensätzen
for dataset in (train_data, dev_data):
    for _, annotations in dataset:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

# 🚀 Schritt 4: Modell-Initialisierung mit allen Daten (nur für Labels!)
def get_examples():
    for text, ann in train_data + dev_data:
        yield Example.from_dict(nlp.make_doc(text), ann)

optimizer = nlp.resume_training()


# 🏋️ Schritt 5: Training (nur auf Trainingsdaten)
n_iter = 20
for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    batches = minibatch(train_data, size=8)
    for batch in batches:
        examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in batch]
        nlp.update(examples, drop=0.35, losses=losses)

    print(f"🔁 Iteration {i+1}/{n_iter}, Loss: {losses['ner']:.4f}")

# 💾 Schritt 6: Modell speichern
output_dir = Path("custom_spacy_model_new")
output_dir.mkdir(exist_ok=True)
nlp.to_disk(output_dir)
print(f"\n✅ Modell gespeichert unter: {output_dir.resolve()}")

# 🔍 Schritt 7: Modell laden und auf dev_data testen
nlp2 = spacy.load(output_dir)

print("\n📊 Evaluation auf dev_data:")
for text, _ in random.sample(dev_data, min(5, len(dev_data))):  # max. 5 Beispiele
    doc = nlp2(text)
    print(f"\n> {text}")
    for ent in doc.ents:
        print(f"  - {ent.text} ({ent.label_})")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 43.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
📥 Trainingsbeispiele: 96, Dev-Beispiele: 25


/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sehr geehrte Damen und Herren,
wie mit Ihrem Kolle..." with entities "[(177, 183, 'VORNAME'), (184, 192, 'NACHNAME'), (2...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sehr geehrte Damen und Herren,
nach Tarifwechsel S..." with entities "[(164, 170, 'VORNAME'), (171, 183, 'NACHNAME'), (1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


🔁 Iteration 1/20, Loss: 1108.7584
🔁 Iteration 2/20, Loss: 999.4371
🔁 Iteration 3/20, Loss: 809.8356
🔁 Iteration 4/20, Loss: 728.1325
🔁 Iteration 5/20, Loss: 565.4085
🔁 Iteration 6/20, Loss: 453.6583
🔁 Iteration 7/20, Loss: 397.7180
🔁 Iteration 8/20, Loss: 516.4252
🔁 Iteration 9/20, Loss: 376.2762
🔁 Iteration 10/20, Loss: 340.9275
🔁 Iteration 11/20, Loss: 400.3647
🔁 Iteration 12/20, Loss: 326.5828
🔁 Iteration 13/20, Loss: 285.6752
🔁 Iteration 14/20, Loss: 325.3445
🔁 Iteration 15/20, Loss: 304.2646
🔁 Iteration 16/20, Loss: 277.3331
🔁 Iteration 17/20, Loss: 247.3439
🔁 Iteration 18/20, Loss: 344.3083
🔁 Iteration 19/20, Loss: 295.9506
🔁 Iteration 20/20, Loss: 305.7312

✅ Modell gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/notebooks/3_model_training_and_testing/spacy_pipeline/custom_spacy_model_new

📊 Evaluation auf dev_data:

> Betreff: Fristlose Kündigung meines Stromvertrags
Sehr geehrte Da

In [5]:
from spacy.scorer import Scorer
from spacy.training import Example
from collections import defaultdict
import pandas as pd

def evaluate_model(nlp, examples):
    scorer = Scorer()
    example_list = []

    for text, ann in examples:
        doc = nlp(text)
        example = Example.from_dict(doc, ann)
        example_list.append(example)

    scores = scorer.score(example_list)

    # Strukturieren der Scores für jede Entitätskategorie
    results = []
    for label, metrics in scores["ents_per_type"].items():
        results.append({
            "Label": label,
            "Precision": round(metrics["p"] * 100, 2),
            "Recall": round(metrics["r"] * 100, 2),
            "F1-Score": round(metrics["f"] * 100, 2)
        })

    return pd.DataFrame(results)

In [6]:
results_df = evaluate_model(nlp, dev_data)
display(results_df)

,Label,Precision,Recall,F1-Score
0,VERTRAGSNUMMER,54.84,89.47,68.00
1,DATUM,57.14,70.59,63.16
2,VORNAME,90.32,90.32,90.32
3,NACHNAME,96.97,91.43,94.12
4,ZÄHLERNUMMER,0.00,0.00,0.00
5,STRASSE,66.67,88.89,76.19
6,GESENDET_MIT,0.00,0.00,0.00
7,HAUSNUMMER,77.78,77.78,77.78
8,POSTLEITZAHL,88.89,100.00,94.12
9,WOHNORT,85.71,75.00,80.00
